
### SCRAPING FROM REDDIT

Questo script fa scraping da reddit in prima istanza per ottenere gli **id** dei vari post appartenenti ad uno dei seguenti subreddit:

- https://www.reddit.com/r/CrohnsDisease/
- https://www.reddit.com/r/UlcerativeColitis/
- https://www.reddit.com/r/IBD/
- https://www.reddit.com/r/ibs/

Per ottenere gli id si definiscono dei periodi temporali sotto forma di `tuple (anno, mese)`, dunque per ogni subreddit si ottengono gli id delle submission inerenti ad un dato periodo temporale utilizzando le api di Reddit. Essi vengono salvati in csv denominati in base al periodo temporale di riferimento e al subreddit.

scraping da 2022-03-01 a 2022-04-01 sul subreddit 'IBD' sarà denominato `IBM_2022-03_submissions.csv`

Successivamente si uniscono tutti gli id in unico file .csv aggiungendo il campo 'subreddit' ad ogni entry.

Come ultima cosa si vanno ad ottenere i dati di ogni submission (titolo, autore, body ecc.) ed i relativi commenti.

***

- [x] Ottenere gli id 
- [x] Ottenere le submissions
- [ ] Ottenere i commenti di primo livello

In [1]:
import praw
from psaw import PushshiftAPI

import datetime as dt
from tqdm import tqdm

import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

from os import listdir
from os.path import isfile, join
import csv

from tqdm.notebook import tqdm,tnrange
import pickle

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS

In [9]:
import import_ipynb
from reddit_access import client_id, client_secret, password, user_agent, username

#PSAW TO RETRIEVE IDs OF SUBMISSIONS
api = PushshiftAPI()

#PRAW TO RETRIEVE ACTUAL CONVERSATIONS
reddit = praw.Reddit(client_id=client_id, 
                     client_secret=client_secret,
                     password=password, 
                     user_agent=user_agent,
                     username=username)

reddit.user.me()

Redditor(name='micheledinelli')

In [ ]:
# DEFINE YEAR-MONTH RANGES TO LOOK UP 
today = dt.datetime.today()

today_month = today.month
today_year = today.year

start_year = 2020
start_month = 12
dates = []
while start_year <= today_year:
    
    if start_year == today_year and start_month > today_month:
        break
        
    dates.append((start_year, start_month))
    
    start_month += 1
    if start_month > 12:
        start_year += 1
        start_month = 1

In [ ]:
def retrieve_ids_in_range(year, month, SR, verbose=True):
    print("Scraping", "r/" + SR, "for:", year, "-", month)
    
    # Seconds in a day/hour
    DAY = 60 * 60 * 24
    HOUR = 60 * 60
    
    start = int(dt.datetime(year, month, 1).timestamp())
    if month < 12:
        end = int(dt.datetime(year, month + 1, 1).timestamp())
    else: 
        end = int(dt.datetime(year + 1, 1, 1).timestamp())
    
    start_epoch = start
    end_epoch = start + DAY 
    
    # Search IDs on a weekly basis
    ids = []
    while end_epoch <= end:
        res = list(api.search_submissions(
                            after = start_epoch,
                            before= end_epoch + HOUR, 
                            subreddit = SR,
                            limit = 100))
        
        print(dt.datetime.fromtimestamp(start_epoch))
        print(dt.datetime.fromtimestamp(end_epoch), '\n')
       
        if verbose:
            print("FROM: ", np.intc(start_epoch).astype("datetime64[s]"), 
                  "TO:", np.intc(end_epoch).astype("datetime64[s]"))        
            print("FIRST: ", np.intc(res[-1].created_utc).astype("datetime64[s]"), 
                  "LAST: ", np.intc(res[0].created_utc).astype("datetime64[s]"))        
            print("number of posts: ", len(res))
        
        for r in res:
            ids.append(r.id)
        
        start_epoch = end_epoch
        end_epoch = start_epoch + DAY
    
    print("SAVING...", SR + "_" + str(year) + "-" + str(month) + "_submissions.csv")
    print(len(ids), " POSTS", "\n")
    pd.DataFrame(ids).drop_duplicates().to_csv("./submissions/" + SR + "_" + str(year) + "-" + str(month) + "_submissions.csv")
    
    return ids

In [3]:
subreddits = ['CrohnsDisease', 'UlcerativeColitis', 'IBD', 'ibs']

In [5]:
for subreddit in subreddits:
    for year, month in dates:
        retrieve_ids_in_range(year = year, month = month, SR = subreddit, verbose = False)

Scraping r/CrohnsDisease for: 2020 - 12
2020-12-01 00:00:00
2020-12-02 00:00:00 

2020-12-02 00:00:00
2020-12-03 00:00:00 

2020-12-03 00:00:00
2020-12-04 00:00:00 

2020-12-04 00:00:00
2020-12-05 00:00:00 

2020-12-05 00:00:00
2020-12-06 00:00:00 

2020-12-06 00:00:00
2020-12-07 00:00:00 

2020-12-07 00:00:00
2020-12-08 00:00:00 

2020-12-08 00:00:00
2020-12-09 00:00:00 

2020-12-09 00:00:00
2020-12-10 00:00:00 

2020-12-10 00:00:00
2020-12-11 00:00:00 

2020-12-11 00:00:00
2020-12-12 00:00:00 

2020-12-12 00:00:00
2020-12-13 00:00:00 

2020-12-13 00:00:00
2020-12-14 00:00:00 

2020-12-14 00:00:00
2020-12-15 00:00:00 

2020-12-15 00:00:00
2020-12-16 00:00:00 

2020-12-16 00:00:00
2020-12-17 00:00:00 

2020-12-17 00:00:00
2020-12-18 00:00:00 

2020-12-18 00:00:00
2020-12-19 00:00:00 

2020-12-19 00:00:00
2020-12-20 00:00:00 

2020-12-20 00:00:00
2020-12-21 00:00:00 

2020-12-21 00:00:00
2020-12-22 00:00:00 

2020-12-22 00:00:00
2020-12-23 00:00:00 

2020-12-23 00:00:00
2020-12-24 00:00

C:\Users\mic\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\mic\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


2021-03-28 00:00:00
2021-03-29 01:00:00 

2021-03-29 01:00:00
2021-03-30 01:00:00 

2021-03-30 01:00:00
2021-03-31 01:00:00 

SAVING... CrohnsDisease_2021-3_submissions.csv
600  POSTS 

Scraping r/CrohnsDisease for: 2021 - 4
2021-04-01 00:00:00
2021-04-02 00:00:00 

2021-04-02 00:00:00
2021-04-03 00:00:00 

2021-04-03 00:00:00
2021-04-04 00:00:00 

2021-04-04 00:00:00
2021-04-05 00:00:00 

2021-04-05 00:00:00
2021-04-06 00:00:00 

2021-04-06 00:00:00
2021-04-07 00:00:00 

2021-04-07 00:00:00
2021-04-08 00:00:00 

2021-04-08 00:00:00
2021-04-09 00:00:00 

2021-04-09 00:00:00
2021-04-10 00:00:00 

2021-04-10 00:00:00
2021-04-11 00:00:00 

2021-04-11 00:00:00
2021-04-12 00:00:00 

2021-04-12 00:00:00
2021-04-13 00:00:00 

2021-04-13 00:00:00
2021-04-14 00:00:00 

2021-04-14 00:00:00
2021-04-15 00:00:00 

2021-04-15 00:00:00
2021-04-16 00:00:00 

2021-04-16 00:00:00
2021-04-17 00:00:00 

2021-04-17 00:00:00
2021-04-18 00:00:00 

2021-04-18 00:00:00
2021-04-19 00:00:00 

2021-04-19 00:00:00

2021-09-26 00:00:00
2021-09-27 00:00:00 

2021-09-27 00:00:00
2021-09-28 00:00:00 

2021-09-28 00:00:00
2021-09-29 00:00:00 

2021-09-29 00:00:00
2021-09-30 00:00:00 

2021-09-30 00:00:00
2021-10-01 00:00:00 

SAVING... CrohnsDisease_2021-9_submissions.csv
989  POSTS 

Scraping r/CrohnsDisease for: 2021 - 10
2021-10-01 00:00:00
2021-10-02 00:00:00 

2021-10-02 00:00:00
2021-10-03 00:00:00 

2021-10-03 00:00:00
2021-10-04 00:00:00 

2021-10-04 00:00:00
2021-10-05 00:00:00 

2021-10-05 00:00:00
2021-10-06 00:00:00 

2021-10-06 00:00:00
2021-10-07 00:00:00 

2021-10-07 00:00:00
2021-10-08 00:00:00 

2021-10-08 00:00:00
2021-10-09 00:00:00 

2021-10-09 00:00:00
2021-10-10 00:00:00 

2021-10-10 00:00:00
2021-10-11 00:00:00 

2021-10-11 00:00:00
2021-10-12 00:00:00 

2021-10-12 00:00:00
2021-10-13 00:00:00 

2021-10-13 00:00:00
2021-10-14 00:00:00 

2021-10-14 00:00:00
2021-10-15 00:00:00 

2021-10-15 00:00:00
2021-10-16 00:00:00 

2021-10-16 00:00:00
2021-10-17 00:00:00 

2021-10-17 00:00:0

2022-03-26 00:00:00
2022-03-27 00:00:00 

2022-03-27 00:00:00
2022-03-28 01:00:00 

2022-03-28 01:00:00
2022-03-29 01:00:00 

2022-03-29 01:00:00
2022-03-30 01:00:00 

2022-03-30 01:00:00
2022-03-31 01:00:00 

SAVING... CrohnsDisease_2022-3_submissions.csv
1168  POSTS 

Scraping r/CrohnsDisease for: 2022 - 4
2022-04-01 00:00:00
2022-04-02 00:00:00 

2022-04-02 00:00:00
2022-04-03 00:00:00 

2022-04-03 00:00:00
2022-04-04 00:00:00 

2022-04-04 00:00:00
2022-04-05 00:00:00 

2022-04-05 00:00:00
2022-04-06 00:00:00 

2022-04-06 00:00:00
2022-04-07 00:00:00 

2022-04-07 00:00:00
2022-04-08 00:00:00 

2022-04-08 00:00:00
2022-04-09 00:00:00 

2022-04-09 00:00:00
2022-04-10 00:00:00 

2022-04-10 00:00:00
2022-04-11 00:00:00 

2022-04-11 00:00:00
2022-04-12 00:00:00 

2022-04-12 00:00:00
2022-04-13 00:00:00 

2022-04-13 00:00:00
2022-04-14 00:00:00 

2022-04-14 00:00:00
2022-04-15 00:00:00 

2022-04-15 00:00:00
2022-04-16 00:00:00 

2022-04-16 00:00:00
2022-04-17 00:00:00 

2022-04-17 00:00:0

...

2022-01-30 00:00:00
2022-01-31 00:00:00 

2022-01-31 00:00:00
2022-02-01 00:00:00 

SAVING... IBD_2022-1_submissions.csv
173  POSTS 

Scraping r/IBD for: 2022 - 2
2022-02-01 00:00:00
2022-02-02 00:00:00 

2022-02-02 00:00:00
2022-02-03 00:00:00 

2022-02-03 00:00:00
2022-02-04 00:00:00 

2022-02-04 00:00:00
2022-02-05 00:00:00 

2022-02-05 00:00:00
2022-02-06 00:00:00 

2022-02-06 00:00:00
2022-02-07 00:00:00 

2022-02-07 00:00:00
2022-02-08 00:00:00 

2022-02-08 00:00:00
2022-02-09 00:00:00 

2022-02-09 00:00:00
2022-02-10 00:00:00 

2022-02-10 00:00:00
2022-02-11 00:00:00 

2022-02-11 00:00:00
2022-02-12 00:00:00 

2022-02-12 00:00:00
2022-02-13 00:00:00 

2022-02-13 00:00:00
2022-02-14 00:00:00 

2022-02-14 00:00:00
2022-02-15 00:00:00 

2022-02-15 00:00:00
2022-02-16 00:00:00 

2022-02-16 00:00:00
2022-02-17 00:00:00 

2022-02-17 00:00:00
2022-02-18 00:00:00 

2022-02-18 00:00:00
2022-02-19 00:00:00 

2022-02-19 00:00:00
2022-02-20 00:00:00 

2022-02-20 00:00:00
2022-02-21 00:00:00

2022-08-01 00:00:00
2022-08-02 00:00:00 

2022-08-02 00:00:00
2022-08-03 00:00:00 

2022-08-03 00:00:00
2022-08-04 00:00:00 

2022-08-04 00:00:00
2022-08-05 00:00:00 

2022-08-05 00:00:00
2022-08-06 00:00:00 

2022-08-06 00:00:00
2022-08-07 00:00:00 

2022-08-07 00:00:00
2022-08-08 00:00:00 

2022-08-08 00:00:00
2022-08-09 00:00:00 

2022-08-09 00:00:00
2022-08-10 00:00:00 

2022-08-10 00:00:00
2022-08-11 00:00:00 

2022-08-11 00:00:00
2022-08-12 00:00:00 

2022-08-12 00:00:00
2022-08-13 00:00:00 

2022-08-13 00:00:00
2022-08-14 00:00:00 

2022-08-14 00:00:00
2022-08-15 00:00:00 

2022-08-15 00:00:00
2022-08-16 00:00:00 

2022-08-16 00:00:00
2022-08-17 00:00:00 

2022-08-17 00:00:00
2022-08-18 00:00:00 

2022-08-18 00:00:00
2022-08-19 00:00:00 

2022-08-19 00:00:00
2022-08-20 00:00:00 

2022-08-20 00:00:00
2022-08-21 00:00:00 

2022-08-21 00:00:00
2022-08-22 00:00:00 

2022-08-22 00:00:00
2022-08-23 00:00:00 

2022-08-23 00:00:00
2022-08-24 00:00:00 

2022-08-24 00:00:00
2022-08-25 00:

2021-04-04 00:00:00
2021-04-05 00:00:00 

2021-04-05 00:00:00
2021-04-06 00:00:00 

2021-04-06 00:00:00
2021-04-07 00:00:00 

2021-04-07 00:00:00
2021-04-08 00:00:00 

2021-04-08 00:00:00
2021-04-09 00:00:00 

2021-04-09 00:00:00
2021-04-10 00:00:00 

2021-04-10 00:00:00
2021-04-11 00:00:00 

2021-04-11 00:00:00
2021-04-12 00:00:00 

2021-04-12 00:00:00
2021-04-13 00:00:00 

2021-04-13 00:00:00
2021-04-14 00:00:00 

2021-04-14 00:00:00
2021-04-15 00:00:00 

2021-04-15 00:00:00
2021-04-16 00:00:00 

2021-04-16 00:00:00
2021-04-17 00:00:00 

2021-04-17 00:00:00
2021-04-18 00:00:00 

2021-04-18 00:00:00
2021-04-19 00:00:00 

2021-04-19 00:00:00
2021-04-20 00:00:00 

2021-04-20 00:00:00
2021-04-21 00:00:00 

2021-04-21 00:00:00
2021-04-22 00:00:00 

2021-04-22 00:00:00
2021-04-23 00:00:00 

2021-04-23 00:00:00
2021-04-24 00:00:00 

2021-04-24 00:00:00
2021-04-25 00:00:00 

2021-04-25 00:00:00
2021-04-26 00:00:00 

2021-04-26 00:00:00
2021-04-27 00:00:00 

2021-04-27 00:00:00
2021-04-28 00:

2021-10-05 00:00:00
2021-10-06 00:00:00 

2021-10-06 00:00:00
2021-10-07 00:00:00 

2021-10-07 00:00:00
2021-10-08 00:00:00 

2021-10-08 00:00:00
2021-10-09 00:00:00 

2021-10-09 00:00:00
2021-10-10 00:00:00 

2021-10-10 00:00:00
2021-10-11 00:00:00 

2021-10-11 00:00:00
2021-10-12 00:00:00 

2021-10-12 00:00:00
2021-10-13 00:00:00 

2021-10-13 00:00:00
2021-10-14 00:00:00 

2021-10-14 00:00:00
2021-10-15 00:00:00 

2021-10-15 00:00:00
2021-10-16 00:00:00 

2021-10-16 00:00:00
2021-10-17 00:00:00 

2021-10-17 00:00:00
2021-10-18 00:00:00 

2021-10-18 00:00:00
2021-10-19 00:00:00 

2021-10-19 00:00:00
2021-10-20 00:00:00 

2021-10-20 00:00:00
2021-10-21 00:00:00 

2021-10-21 00:00:00
2021-10-22 00:00:00 

2021-10-22 00:00:00
2021-10-23 00:00:00 

2021-10-23 00:00:00
2021-10-24 00:00:00 

2021-10-24 00:00:00
2021-10-25 00:00:00 

2021-10-25 00:00:00
2021-10-26 00:00:00 

2021-10-26 00:00:00
2021-10-27 00:00:00 

2021-10-27 00:00:00
2021-10-28 00:00:00 

2021-10-28 00:00:00
2021-10-29 00:

2022-04-08 00:00:00
2022-04-09 00:00:00 

2022-04-09 00:00:00
2022-04-10 00:00:00 

2022-04-10 00:00:00
2022-04-11 00:00:00 

2022-04-11 00:00:00
2022-04-12 00:00:00 

2022-04-12 00:00:00
2022-04-13 00:00:00 

2022-04-13 00:00:00
2022-04-14 00:00:00 

2022-04-14 00:00:00
2022-04-15 00:00:00 

2022-04-15 00:00:00
2022-04-16 00:00:00 

2022-04-16 00:00:00
2022-04-17 00:00:00 

2022-04-17 00:00:00
2022-04-18 00:00:00 

2022-04-18 00:00:00
2022-04-19 00:00:00 

2022-04-19 00:00:00
2022-04-20 00:00:00 

2022-04-20 00:00:00
2022-04-21 00:00:00 

2022-04-21 00:00:00
2022-04-22 00:00:00 

2022-04-22 00:00:00
2022-04-23 00:00:00 

2022-04-23 00:00:00
2022-04-24 00:00:00 

2022-04-24 00:00:00
2022-04-25 00:00:00 

2022-04-25 00:00:00
2022-04-26 00:00:00 

2022-04-26 00:00:00
2022-04-27 00:00:00 

2022-04-27 00:00:00
2022-04-28 00:00:00 

2022-04-28 00:00:00
2022-04-29 00:00:00 

2022-04-29 00:00:00
2022-04-30 00:00:00 

2022-04-30 00:00:00
2022-05-01 00:00:00 

SAVING... ibs_2022-4_submissions.c

In [4]:
mypath = "./submissions/"
files = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

merged_file = []
for filename in files:
    with open(filename, 'r') as csv_file:
        file = csv.reader(csv_file)
        for row in file:
            merged_file.append(row)
            
merged_df = pd.DataFrame(merged_file, columns=['index', 'id'])
merged_df.drop(columns=['index'], inplace=True)
merged_df.to_csv('all_ids.csv')

In [5]:
import os 

all_ids = pd.read_csv('all_ids.csv')
all_ids.drop(columns=['Unnamed: 0'], inplace=True)

y = []
for subreddit in subreddits:
    x = []
    for fname in os.listdir('./submissions/'):
        if subreddit in fname:
            x.append(pd.read_csv(join('./submissions/', fname),index_col=0))
    
    x = pd.concat(x)
    x['subreddit'] = subreddit
    y.append(x)

y = pd.concat(y)
y.to_csv('all_ids_withsr.csv')

In [6]:
all_ids_withsr = pd.read_csv('all_ids_withsr.csv')
all_ids_withsr.drop(columns = ['Unnamed: 0'], inplace = True)
all_ids_withsr.drop_duplicates(inplace = True)
all_ids_withsr.groupby('subreddit').count()

0
subreddit               
CrohnsDisease      21475
IBD                 3908
UlcerativeColitis  16680
ibs                36583

In [22]:
ibd_ids = all_ids_withsr[all_ids_withsr['subreddit'] == 'IBD'].reset_index()
ibd_ids.rename(columns = {'0': 'id'}, inplace = True)

ibd_posts = []
for id_ in tqdm (ibd_ids['id'], desc="Loading from subreddit IBD"):
    post = reddit.submission(id_)
    ibd_posts.append([post.title, post.author, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    
df_ibd = pd.DataFrame(ibd_posts, columns = ['title', 'author', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
df_ibd['created'] = df_ibd['created'].apply(lambda x: dt.datetime.fromtimestamp(x))

df_ibd.to_csv('./submissions_scraped/ibd_submissions.csv')
df_ibd.head(10)

Loading from subreddit IBD:   0%|          | 0/3908 [00:00<?, ?it/s]

title             author  \
0  Anyone else get really intense cramps in the ....               None   
1  Been off Accutane for 2 years, have had consti...    OptimalSyrup193   
2                   Anyone on a JAK inhibitor trial?  abundance-and-joy   
3  Has anyone experienced this? Symptoms changing...    rachelcalabresi   
4  [FYI] Crohns & Colitis Foundation - #GivingTue...     visualoptimism   
5                       Does IBD affect stomach fat?           Kurtinho   
6                   Survey Results and Videos on IBD          GISociety   
7            Head feels weird after Stelara infusion   MarshmallowCat14   
8  Just began taking 25mg of Methotrexate once a ...               None   
9                                  [deleted by user]               None   

   score      id subreddit                                                url  \
0     22  k4wsmz       IBD                                                      
1      1  k4wjyp       IBD  https://www.reddit.com/r/IBD/comments/k4wjyp/b...   
2      7  k4vqri       IBD  https://www.reddit.com/r/IBD/comments/k4vqri/a...   
3     10  k4vpft       IBD  https://www.reddit.com/r/IBD/comments/k4vpft/h...   
4     17  k4njtg       IBD  https://online.crohnscolitisfoundation.org/sit...   
5      7  k4gu2a       IBD  https://www.reddit.com/r/IBD/comments/k4gu2a/d...   
6      4  k5k88f       IBD  https://www.reddit.com/r/IBD/comments/k5k88f/s...   
7      7  k5ir5w       IBD  https://www.reddit.com/r/IBD/comments/k5ir5w/h...   
8      7  k5gey6       IBD                                                      
9     40  k5fwow       IBD                                                      

   num_comments                                               body  \
0             7                                          [deleted]   
1             0  I finally had enough and decides to go to a ga...   
2             7  Doc is recommending JAK inhibitor to treat Cro...   
3             5  I hope this message doesn’t upset anyone in th...   
4             3                                                      
5            13  I’m 22, been diagnosed since 17 and since I’ve...   
6             0  Last year, I posted in this community about a ...   
7            13  I posted a longer post in the UC sub, but thou...   
8             3                                          [deleted]   
9             2                                          [removed]   

              created  
0 2020-12-02 00:20:31  
1 2020-12-02 00:07:39  
2 2020-12-01 23:26:45  
3 2020-12-01 23:24:59  
4 2020-12-01 16:56:21  
5 2020-12-01 09:14:49  
6 2020-12-02 23:36:58  
7 2020-12-02 22:22:35  
8 2020-12-02 20:31:08  
9 2020-12-02 20:07:36

In [25]:
ulc_col_ids = all_ids_withsr[all_ids_withsr['subreddit'] == 'UlcerativeColitis'].reset_index()
ulc_col_ids.rename(columns = {'0': 'id'}, inplace = True)

ulc_col_posts = []
for id_ in tqdm (ulc_col_ids['id'], desc="Loading from subreddit Ulcerative Colitis"):
    post = reddit.submission(id_)
    ulc_col_posts.append([post.title, post.author ,post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    
df_ulc_col = pd.DataFrame(ulc_col_posts, columns = ['title', 'author', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
df_ulc_col['created'] = df_ulc_col['created'].apply(lambda x: dt.datetime.fromtimestamp(x))

df_ulc_col.to_csv('./submissions_scraped/ulc_col_submissions.csv')
df_ulc_col.head(10)

Loading from subreddit Ulcerative Colitis:   0%|          | 0/16680 [00:00<?, ?it/s]

title            author  score  \
0                                          Berberine             iserd      2   
1  Has anyone experienced this? Symptoms changing...   rachelcalabresi      2   
2                                  [deleted by user]              None      2   
3                      running to the toilet be like       wheelieboii    142   
4                                  Mesalamine Enemas           SS13908      3   
5                                   Elimination diet              None      3   
6  Mesalamine side effect? Head feels hot at time...  Throwawayhippo12      5   
7           Ulcerative colitis and suicidal thoughts    colitispatient     74   
8                                Supplement question  BreakfastExpress      2   
9  [FYI] Crohn's & Colitis Foundation #GivingTues...    visualoptimism     10   

       id          subreddit  \
0  k4vtmp  UlcerativeColitis   
1  k4vq6z  UlcerativeColitis   
2  k4uvuj  UlcerativeColitis   
3  k4uceq  UlcerativeColitis   
4  k4rnfg  UlcerativeColitis   
5  k4pz3y  UlcerativeColitis   
6  k4oz78  UlcerativeColitis   
7  k4mysk  UlcerativeColitis   
8  k4myh4  UlcerativeColitis   
9  k4mxqs  UlcerativeColitis   

                                                 url  num_comments  \
0  https://www.reddit.com/r/UlcerativeColitis/com...             3   
1  https://www.reddit.com/r/UlcerativeColitis/com...             5   
2                                                                5   
3                    https://v.redd.it/qefc6j7e6n261             7   
4  https://www.reddit.com/r/UlcerativeColitis/com...             9   
5                                                               11   
6  https://www.reddit.com/r/UlcerativeColitis/com...             8   
7  https://www.reddit.com/r/UlcerativeColitis/com...            68   
8  https://www.reddit.com/r/UlcerativeColitis/com...             9   
9  https://online.crohnscolitisfoundation.org/sit...             0   

                                                body             created  
0  Hi all. I am in remission currently taking mes... 2020-12-01 23:30:39  
1  I hope this message doesn’t upset anyone in th... 2020-12-01 23:26:03  
2                                          [removed] 2020-12-01 22:43:52  
3                                                    2020-12-01 22:17:28  
4  I just took these enemas for four weeks to cal... 2020-12-01 20:08:55  
5                                          [deleted] 2020-12-01 18:50:45  
6  As the title says I’ve noticed that my forehea... 2020-12-01 18:04:27  
7  Hey everyone, i was diagnosed with ulcerative ... 2020-12-01 16:27:51  
8  Has anyone tried indigo naturalis/ qing dai? I... 2020-12-01 16:27:24  
9                                                    2020-12-01 16:26:23

In [26]:
crohns_disease_ids = all_ids_withsr[all_ids_withsr['subreddit'] == 'CrohnsDisease'].reset_index()
crohns_disease_ids.rename(columns = {'0': 'id'}, inplace = True)

crohns_disease_posts = []
for id_ in tqdm (crohns_disease_ids['id'], desc="Loading from subreddit CrohnsDisease"):
    post = reddit.submission(id_)
    crohns_disease_posts.append([post.title, post.author, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    
df_crohns_disease = pd.DataFrame(crohns_disease_posts, columns = ['title', 'author', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
df_crohns_disease['created'] = df_crohns_disease['created'].apply(lambda x: dt.datetime.fromtimestamp(x))

df_crohns_disease.to_csv('./submissions_scraped/crohns_disease_submissions.csv')
df_crohns_disease.head(10)

Loading from subreddit CrohnsDisease:   0%|          | 0/21475 [00:00<?, ?it/s]

title               author  \
0  Ibd awareness week! Crohns and colitis website...                 None   
1    Not diagnosed. Looking for advice while waiting                 None   
2                                    Painkillers....          ChuckFiasco   
3  Anyone try the trial for JAK inhibitor to trea...    abundance-and-joy   
4  Why is it so complicated just to get treatment...        VurucaAssault   
5                                  Intermittent FMLA            Tmoore188   
6                               Colonoscopy Findings         cheetoPalmer   
7                               DAE Bolt their food?           AJClarkson   
8                                     Chalky stools?  Constant-gardener89   
9                   Chron’s fever - is it avoidable?                MCHFA   

   score      id      subreddit  \
0     48  k4xgh0  CrohnsDisease   
1      2  k4x6p1  CrohnsDisease   
2      3  k4w6h7  CrohnsDisease   
3      3  k4vof1  CrohnsDisease   
4      9  k4vim5  CrohnsDisease   
5      1  k4t4xh  CrohnsDisease   
6      2  k4rodi  CrohnsDisease   
7      1  k4riy4  CrohnsDisease   
8      1  k4qapv  CrohnsDisease   
9      1  k4px8u  CrohnsDisease   

                                                 url  num_comments  \
0                                                                6   
1                                                                2   
2  https://www.reddit.com/r/CrohnsDisease/comment...            13   
3  https://www.reddit.com/r/CrohnsDisease/comment...             5   
4  https://www.reddit.com/r/CrohnsDisease/comment...             6   
5  https://www.reddit.com/r/CrohnsDisease/comment...             4   
6  https://www.reddit.com/r/CrohnsDisease/comment...             5   
7  https://www.reddit.com/r/CrohnsDisease/comment...             1   
8  https://www.reddit.com/r/CrohnsDisease/comment...             9   
9  https://www.reddit.com/r/CrohnsDisease/comment...             2   

                                                body             created  
0                                          [deleted] 2020-12-02 00:57:04  
1                                          [deleted] 2020-12-02 00:42:02  
2  I’ve been battling a flare for the better part... 2020-12-01 23:48:33  
3  Doc recommending JAK inhibitor and looking for... 2020-12-01 23:23:30  
4  I was dx in 2009 and it has never been a simpl... 2020-12-01 23:15:18  
5  I am initiating the process of applying for in... 2020-12-01 21:18:35  
6  I was diagnosed with Crohn’s Disease in 2016. ... 2020-12-01 20:10:09  
7  For a long time, my gag reflex was on a hair t... 2020-12-01 20:03:18  
8  Has anyone else experienced this? Instead of b... 2020-12-01 19:05:36  
9  Dearest community,\n\nI hope you’re all doing ... 2020-12-01 18:48:16

In [7]:
ibs_ids = all_ids_withsr[all_ids_withsr['subreddit'] == 'ibs'].reset_index()
ibs_ids.rename(columns = {'0': 'id'}, inplace = True)

ibs_posts = []
for id_ in tqdm (ibs_ids['id'], desc="Loading from subreddit ibs"):
    post = reddit.submission(id_)
    ibs_posts.append([post.title, post.author, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    
    
df_ibs = pd.DataFrame(ibs_posts, columns = ['title', 'author', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
df_ibs['created'] = df_ibs['created'].apply(lambda x: dt.datetime.fromtimestamp(x))

df_ibs.to_csv('./submissions_scraped/ibs_submissions.csv')
df_ibs.head(10)

Loading from subreddit ibs:   0%|          | 0/36583 [00:00<?, ?it/s]

title                author  \
0                                Align Probiotic Gas     Complete_Lack4089   
1        Does chik-fil-a hurt anyone else’s stomach?                  None   
2  Has any one found the whole30 diet helps? I us...               Lordus_   
3  Can anyone with IBS and constipation give advice?                  None   
4                                                  🍇                  None   
5                                          Relatable                  None   
6    How to gain weight with all these restrictions?             rlopes528   
7                                Help, Help, Help ):      Sensitive_Bottle   
8                                       Gurgulations  intangible-tangerine   
9                                          Can’t win       georgiaokeeffer   

   score      id subreddit                                                url  \
0      6  k4x6e3       ibs  https://www.reddit.com/r/ibs/comments/k4x6e3/a...   
1      2  k4wdaq       ibs  https://www.reddit.com/r/ibs/comments/k4wdaq/d...   
2      1  k4w98g       ibs  https://www.reddit.com/r/ibs/comments/k4w98g/h...   
3      1  k4w1i5       ibs                                                      
4      1  k4voga       ibs                                                      
5      8  k4ufr1       ibs                                                      
6      3  k4ubao       ibs  https://www.reddit.com/r/ibs/comments/k4ubao/h...   
7      1  k4tuqo       ibs  https://www.reddit.com/r/ibs/comments/k4tuqo/h...   
8     12  k4tgee       ibs  https://www.reddit.com/r/ibs/comments/k4tgee/g...   
9      4  k4t783       ibs  https://www.reddit.com/r/ibs/comments/k4t783/c...   

   num_comments                                               body  \
0             9  Does Align Prebiotic + Probiotic give anyone e...   
1             6  Anytime I eat at this place I get heartburn/re...   
2             0                                                      
3             9                                          [deleted]   
4             4                                          [deleted]   
5             0                                          [deleted]   
6            13  So, I'm on a zero lactose, zero gluten, low su...   
7            11  Hi guys new here and my IBS is actually insane...   
8             0  "Cow's milk and ewe's milk..... is not good fo...   
9             5  Just feeling frustrated and defeated after a c...   

              created  
0 2020-12-02 00:41:35  
1 2020-12-01 23:58:09  
2 2020-12-01 23:52:25  
3 2020-12-01 23:41:28  
4 2020-12-01 23:23:32  
5 2020-12-01 22:22:07  
6 2020-12-01 22:15:51  
7 2020-12-01 21:52:37  
8 2020-12-01 21:33:47  
9 2020-12-01 21:21:39

In [53]:
all_ids = pd.read_csv('all_ids.csv')
all_ids.dropna(inplace=True)

posts = []
for id in all_ids['id']:
    if id != str(0):
        post = reddit.submission(id)
        print(post.title)
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])

Just had a Right Hemicolectmy. Much needed and feeling better already.
2nd dose on entyvio 3 days ago but I have a concern
I (17M) get nauseous and have stomach pain when running/doing cardio
Anxiety affects Crohns?
Anyone been diagnosed with non-specific colitis?
Fertility issues?
Best treatment
Stelara injection issues
Prianal crohn's
Any advice for preparing for surgery?
[deleted by user]
My Crohn's story, I guess
[deleted by user]
[deleted by user]
Did your mental health delay your diagnosis?
In your opinion, what caused your Crohn's?
Biologics Decision
Virtual IBD Patient to Patient Education Project 2021 happens tomorrow!
[deleted by user]
Fistula Abscesses
Very new, symptom questions
[deleted by user]
[deleted by user]
really struggling
Doctor recommended Entyvio early as preventative measure. Do I Do it?
Trouble swallowing and choking on food? Are these symptoms Crohn’s related?
Is apriso even worth it?
MAP vaccine?
Leg cramps every night! On budesonide and sulfasalazine
Mucus 

Surgery advice?
Newly diagnosed. Right lower quadrant pain is sooo far down, between my groin and bladder. Typical?
Running out of medicine options
Prednisone
Headaches.
[deleted by user]
Tips for controlling a flare without meds (See post)
COVID vax and Stelara?
[deleted by user]
[deleted by user]
New crohns questions
Toxic Megacolon? Who's had it and what happens with it?
[deleted by user]
Diagnosed w/ CD but having other symptoms?
Do you go to the ER for an obstruction immediately?
Janssen CarePath
Crohns and constipation?
Chest Pain Natural Treatment - Saaol Heart Hospital Surat
Moviprep day, can't wait...
Anyone else feeling tiredness in stomach?
Sexual dysfunction
Any tips on dealing with stoma leaks?
Red patterning on my skin from Crohns? It comes and goes, doesn’t hurt or itch, and is not raised at all. What gives?
Has anyone gone through law school here?
will colonoscopy catch Crohn's or ulcerative colitis if it's in remission?
Anyone else get fevery/chills at nighttime only?


Newly diagnosed - any advice?
Going to school is awful
Best travel cooler for 6-8 biologic pens for 20hr travel day?
Fistula Surgery
Loud noises
What’s your personal belief on the cause of Crohn’s disease?
Humira not working
[deleted by user]
Gross Bathrooms (rant)
Just got my diagnosis
Anyone here experienced skin issues?
Hello
[deleted by user]
[deleted by user]
azathioprine to maintain remission in Crohn's?
Anyone knows what a CT scan of the abdominal area is used for?
Anyone had any odd experiences with colorectal surgeons?
Newly Diagnosed: Treatment Options (Humira, LDN, Chinese Medicine)
Doctor doubling dose of Humira
Just had my first CT scan and I’m sicker than ever.
Dating someone with Crohns
[deleted by user]
[deleted by user]
Why did my body have low serum levels? I don’t understand
Nutrient absorption/hair loss
First Remicade treatment
My first ever infusion is tomorrow. Pro tips?
Anyone tried everlywell food sensitivity test?
SynoGut Review: Latest OCTOBER Update) GREAT Fo

Bowel pain
No calpro test?
Crohns is killing my teeth
Multiple Anal Fistulas on my first try!
Additional colitis diagnosis
Magnesium citrate or gotlytely?
Im a slave to the bathroom
Have you had inflammation in large and small intestine and or a stricture?
Vamorolone - Potentially a new steroid for crohns patients with less side effects?
Lmao I woke up at the hospital in the middle of the night and had to shit like crazy. Flagyl makes my stomach feel like a legit warzone. Anybody feels me?
[deleted by user]
All food hurts
Hunger pains flare up?
[deleted by user]
Pointers for moving with Crohn’s?
[deleted by user]
Colonoscopy questions
Pain in left lower/mid abdomin
[deleted by user]
[deleted by user]
Entyvio question
Does it ever get better
[deleted by user]
Testicles (definitely a different title)
Is this candida overgrowth ? I’ve had white patches on my tongue all the time with random red patches … wondering why that is .. my skin has been dry burning red lately so I’m wondering if i

why does this happen??
I just managed to give myself my first solo dose of Stelara..
Anyone else experience this?
Anyone have a flare up of mega pain but no running to the bathroom?
Anyone have luck in reducing flares with tea?
Is there such thing as pre-crohns?
How often do you visit your GI
Finally got a preliminary diagnosis and have some questions!
What is non-specific inflammation?
I'm Official!
[deleted by user]
Learning to not blame myself and unwanted advice
Depression when flaring?
Hi all, just got officially diagnosed yesterday
[deleted by user]
Getting Lightened with Crohn's
Paid Study Participants Needed
Fainting after colonoscopy
Humira fungal infection risk
PillCam live images
Methotrexate & acne
Tablet prep exists!
Having colonoscopy in 2 days. Is this prep normal ?
Figured nobody could appreciate this more than my fellow Crohn’s/Colitis fam: (gift from my mom today)
Has anyone here with Crohn's had a contrast barium enema procedure? I'm supposed to get one and I want to

...

[deleted by user]
Long time side effects after prednisone
The one little thing that helps with Crohn's nausea and gastritis is lying down and resting on my left side. Nausea/headache/cramps alway reduce. Hope this helps someone.
Literally starving to death from Crohn’s
Nauseous when eating out
Newly diagnosed and so, so confused.
Work revoking remote accommodation.
Roid Rage on Prednisone Taper
A little Saturday laugh…I will always hear this now 😂
Just a little Saturday Funny…
Please help. I can’t even eat anything without being in so much pain. I feel like I’m dying. I’m waiting on my GI doctor to schedule me. It’s looking like crohns but I’ll have to have a scope done. Should I go to the er? I don’t know what to do.
Having financial nightmares about being able to afford my treatment as the rules have changed again with that American healthcare system.
Fellow Crohnies...I present to you your future financial success. We are gunna be RIIIICH! When finally having a shit disease pays off

Good High Calorie Snacks?
[deleted by user]
Is it still possible to have a bad flare on biologics?
Steroid/dexamethasone bloat
I honestly need help
Rant
[deleted by user]
[deleted by user]
[deleted by user]
[deleted by user]
[deleted by user]
Perianal fistulas with Crohn’s
Remicade While Traveling Abroad?
I hate fatigue so damn much, never goes away had tons of blood tests just feeling weaker....
Someone make a Crohns disease dating app
Any other pain medication or remedies
not looking for medical advice, but wondering if anyone has experienced something similar.
Alternative medicine treatments
9 days in hospital for pancreatitis and hepatitis due to blocked bile duct after gallbladder removal
You ever shit so hard that you get pins and needles from your feet up through your legs?
[deleted by user]
Any ileumless people here have tips on eating oily food?
Crohns Disease and sports
Introducing foods
Intestines scan
Crazy dandruff (maybe psoriasis?) after stopping methotrexate
Anxiety/Dep

what happens if you miss your dose?
[deleted by user]
Is this likely to be crohns ?
Experience with Xeljanz and outdoor living
Tired Of explaining to my Family after 17 Years, Why I'm sick all the time. I have Severe Crohns Disease and Colitis. It's like they want to Blame anything else But The crohns. I currently have an Obstruction, Have had 5 resections, Colon Cancer, Cdiff 4 times and Toxic Megacolon amongst other thing
[deleted by user]
aches and pains
[deleted by user]
Healthcare appeal.,
I haven't eaten a sandwich in years but my bowels have been a bit better lately so I was thinking of trying again - what types of bread/peanut butter/etc do you eat?
For the new mamas here: you can do this!
Celiac Disease or Crohn's Disease?
Fistula pain is unbearable
Psychological Aspects of Crohn's
Yellow skin but eyes normal
I have Crohn’s disease and have a rare manifestation in my labia area with little tiny bumps that won’t go away. I’m extremely insecure of having relationships with guys 

McDonald’s.
Update for my surgery.
[deleted by user]
About to start Stelara
So fucking annoyed of this shit... literally.
[deleted by user]
This is it or so they say
Does anyone get frequent headaches after they started treatment with Humira? Also high blood pressure? I’ve never had headaches but get them frequently and also high blood pressure…is this common?
Has anyone else lost all shyness in public bathrooms...
Having bowel obstructions since November, not sure what to do
Entyvio and sun exposure issues?
Remicade Questions:
[deleted by user]
[deleted by user]
[deleted by user]
Constipation after infusion
[deleted by user]
Meal Replacement Shakes?
Small Business Owners
Have you disclosed your illness to your employer?
Can Crohn’s cause rashes?
Just did my first adalimumab injections; it wasn’t so bad, but I do now feel like a human pincushion…
[deleted by user]
On Stelara, CDC Booster Recommendation for J&J
How can you deal with gazes when you are around freinds.
BCAA (Branch chain 

KeyboardInterrupt: 

In [68]:
df = pd.DataFrame(posts, columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
df['created'] = df['created'].apply(lambda x: dt.datetime.fromtimestamp(x))

df.reset_index()
df.drop_duplicates().to_csv('./submissions/submissions_complete.csv')

a = pd.read_csv('./submissions/submissions_complete.csv')

Unnamed: 0                                              title  score  \
0               0  Just had a Right Hemicolectmy. Much needed and...    176   
1               1  2nd dose on entyvio 3 days ago but I have a co...      1   
2               2  I (17M) get nauseous and have stomach pain whe...      2   
3               3                            Anxiety affects Crohns?     18   
4               4   Anyone been diagnosed with non-specific colitis?      3   
...           ...                                                ...    ...   
13175       13187  About how many hours of sleep do you get on av...      3   
13176       13188  Has anyone here been diagnosed with IBD while ...      2   
13177       13189                 Crohn's disease and contraception?      2   
13178       13190  Setting up repeat delivery/subscription for se...      2   
13179       13191  How's everyone feeling/dealing with the world ...     51   

           id      subreddit  \
0      pziujv  CrohnsDisease   
1      pzir07  CrohnsDisease   
2      pzhu3d  CrohnsDisease   
3      pzfkhy  CrohnsDisease   
4      pzdjjj  CrohnsDisease   
...       ...            ...   
13175  tmvw4u  CrohnsDisease   
13176  tmudwy  CrohnsDisease   
13177  tmsdb4  CrohnsDisease   
13178  tmqu4b  CrohnsDisease   
13179  tmme45  CrohnsDisease   

                                                     url  num_comments  \
0                    https://i.redd.it/85fy2qo82xq71.jpg            28   
1      https://www.reddit.com/r/CrohnsDisease/comment...             4   
2      https://www.reddit.com/r/CrohnsDisease/comment...             3   
3      https://www.reddit.com/r/CrohnsDisease/comment...            13   
4                                                    NaN             2   
...                                                  ...           ...   
13175  https://www.reddit.com/r/CrohnsDisease/comment...             2   
13176  https://www.reddit.com/r/CrohnsDisease/comment...            16   
13177  https://www.reddit.com/r/CrohnsDisease/comment...             0   
13178  https://www.reddit.com/r/CrohnsDisease/comment...             1   
13179  https://www.reddit.com/r/CrohnsDisease/comment...            75   

                                                    body              created  
0                                                    NaN  2021-10-02 00:38:45  
1      So it just so happened to line up that I’m com...  2021-10-02 00:33:16  
2      Does anyone know why this is? I’ve noticed tha...  2021-10-01 23:42:38  
3      Does anybody else get Crohn’s flares or just w...  2021-10-01 21:44:28  
4                                              [deleted]  2021-10-01 20:04:12  
...                                                  ...                  ...  
13175  \n\n[View Poll](https://www.reddit.com/poll/tm...  2022-03-24 19:57:17  
13176  I’m just curious. I have had major GI symptoms...  2022-03-24 19:36:01  
13177                                          [removed]  2022-03-24 19:07:45  
13178  Hey y'all! I'm so sick of running out to diffe...  2022-03-24 18:46:01  
13179  The virus is still there and more contagious t...  2022-03-24 17:41:38  

[13180 rows x 9 columns]